<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_8_AI_Agent_%E1%84%86%E1%85%A1%E1%86%BA%E1%84%87%E1%85%A9%E1%84%80%E1%85%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Agent: 실시간 웹 검색 에이전트 (ReAct:Reasoning + Acting)

## AI 에이전트(AI Agent)란?
단순한 LLM(ChatGPT 등)은 묻는 말에 대답만 할 수 있지만, **에이전트(Agent)**는 스스로 계획을 세우고 **도구(Tool)**를 사용하여 실제 행동을 합니다.

- 에이전트의 핵심 작동 원리:
에이전트는 ReAct (Reasoning + Acting) 방식을 주로 사용합니다.

  - Reasoning (생각/추론): "사용자가 현재 대통령을 물어봤네? 내 학습 데이터는 옛날 거니까 검색 도구를 써야겠다."
  - Acting (행동/도구사용): 실제로 검색(Search) 도구를 실행합니다.
  - Observation (관찰): 검색 결과를 읽어봅니다. "아, 결과에 따르면 현재 대통령은 OOO이구나."
  - Final Answer (최종답변): "현재 한국의 대통령은 OOO입니다."라고 사용자에게 대답합니다.

이 개념을 바탕으로, LangGraph가 자동으로 만들어주는 에이전트를 실습

## ReAct 에이전트란?
**ReAct (Reasoning + Acting)**는 AI가 스스로 "생각(Reasoning)"하고, 필요한 "행동(Acting, 도구 사용)"을 결정하는 패턴입니다.

- LangGraph 맛보기 실습: 노드와 조건부 엣지를 우리가 직접 연결했습니다. (수동)

- 이번 실습: create_react_agent 함수를 사용하여 단 한 줄로 에이전트를 만듭니다. (자동)

이 에이전트는 사용자의 질문에 답하기 위해 인터넷 검색이 필요한지 스스로 판단하고 검색합니다.

In [ ]:
# 1. 라이브러리 설치
!pip install -qU langgraph langchain-openai langchain-community duckduckgo-search ddgs

In [ ]:
import os
import getpass

# 2. OpenAI API Key 설정
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key 입력: ")

OpenAI API Key 입력: ··········


## 검색 도구 및 에이전트 생성
여기서 중요한 점은 그래프를 구성하는 코드가 사라지고, create_react_agent 함수 하나로 대체된다는 점입니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent

# 1. LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 2. 도구(Tool) 정의 - 실제 웹 검색 도구 사용
# DuckDuckGoSearchRun은 API Key 없이 무료로 검색 가능한 도구입니다.: https://start.duckduckgo.com/
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# 3. 미리 만들어진 ReAct 에이전트 생성
# LangGraph create_react_agent가 내부적으로 노드, 엣지, 분기 로직을 모두 자동으로 설정해줍니다.
agent_executor = create_react_agent(llm, tools)

print("---검색 에이전트 생성 완료!")

---검색 에이전트 생성 완료!


/tmp/ipython-input-876092235.py:15: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)


## 에이전트 테스트 (실시간 검색)
LLM은 최신 정보를 모릅니다(예: 오늘 날씨, 최근 뉴스 등). 하지만 이 에이전트는 검색 도구를 사용해 답변해냅니다.

In [ ]:
from langchain_core.messages import HumanMessage

def run_agent(question):
    print(f"\n 질문: {question}")
    print("-" * 50)

    # 에이전트 실행 (스트리밍으로 과정 확인)
    inputs = {"messages": [HumanMessage(content=question)]}

    for chunk in agent_executor.stream(inputs, stream_mode="values"):
        # 메시지의 마지막 내용만 출력
        message = chunk["messages"][-1]

        # 도구가 실행되거나 답변이 생성될 때 출력
        if message.type == "ai":
            # 도구 호출이 포함된 경우
            if message.tool_calls:
                print(f"================[생각] 검색이 필요해! -> '{message.tool_calls[0]['args']}' 검색 시도...")
            # 최종 답변인 경우
            else:
                print(f"================[답변] {message.content}")
        elif message.type == "tool":
            print(f"================[결과] 검색 완료 (내용 일부: {message.content[:100]}...)")

In [ ]:
# --- 테스트 1 실행 ---
# 1. 최신 정보가 필요한 질문 (LLM 학습 데이터에 없는 내용)
run_agent("2025년 11월 현재 주요 뉴스는 뭐야?")


 질문: 2025년 11월 현재 주요 뉴스는 뭐야?
--------------------------------------------------
================[생각] 검색이 필요해! -> '{'query': 'November 2025 current news'}' 검색 시도...
================[결과] 검색 완료 (내용 일부: November 2025 was the eleventh month of the current common year. The month, which began on a Saturda...)
================[답변] 2025년 11월의 주요 뉴스 내용은 다음과 같습니다:

1. **열대성 사이클론 센야르**: 이 강력한 열대성 사이클론과 다른 사이클론 계통들이 동시에 발생하면서 위기가 극에 달했습니다. 북동 몬순과 라니냐 조건이 경합하는 가운데 자연 재해의 영향이 심각했습니다.

2. **글로벌 갈등과 인도적 위기**: 가자 지역에서의 인도적 위기와 함께 전 세계적으로 갈등이 격화되고 있습니다. 이러한 상황은 국제 외교 및 안보 분야에서 중요한 발전을 초래하고 있습니다.

3. **미국의 정치 및 정부 상황**: 미국에서는 선거일 드라마, 정부 셧다운 협상, 그리고 대통령 인터뷰가 주요 뉴스로 보도되었습니다. 

이와 같은 사건들은 2025년 11월의 보도에서 중요한 이슈로 다뤄지고 있습니다.


In [ ]:
# --- 테스트 2 실행 ---
# 2. 일반 상식 질문 (검색 없이 대답 가능한지 확인)
run_agent("사과는 영어로 뭐야?")


 질문: 사과는 영어로 뭐야?
--------------------------------------------------
================[답변] 사과는 영어로 "apple"이라고 합니다.
